In [7]:
import pyomo.environ as pyo
import pyomo.opt as pyopt
import numpy as np

import numpy as np
from scipy.optimize import minimize



# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 100

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Transportation costs from facilities to customers
trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
np.random.seed(42)
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Scenario probabilities
scenario_probs = np.ones(num_scenarios) / num_scenarios

# Create the model
model = pyo.ConcreteModel()

# First stage decision variables: Facility opening
model.open_facility = pyo.Var(range(num_facilities), within=pyo.Binary)

# Second stage decision variables: Distribution amounts
model.distribute = pyo.Var(range(num_facilities), range(num_customers), range(num_scenarios), within=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    fixed_cost = sum(fixed_costs[i] * model.open_facility[i] for i in range(num_facilities))
    transport_cost = sum(scenario_probs[s] * sum(trans_costs[i][j] * model.distribute[i,j,s] 
                         for i in range(num_facilities) for j in range(num_customers))
                         for s in range(num_scenarios))
    return fixed_cost + transport_cost

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints

# Capacity constraints
def capacity_rule(model, i, s):
    return sum(model.distribute[i,j,s] for j in range(num_customers)) <= capacities[i] * model.open_facility[i]

model.capacity_const = pyo.Constraint(range(num_facilities), range(num_scenarios), rule=capacity_rule)

# Demand satisfaction constraints
def demand_rule(model, j, s):
    return sum(model.distribute[i,j,s] for i in range(num_facilities)) >= demand_scenarios[s][j]

model.demand_const = pyo.Constraint(range(num_customers), range(num_scenarios), rule=demand_rule)

# Solve the model
solver = pyopt.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Optimal objective value:", pyo.value(model.obj))
print("\nFacilities to open:")
for i in range(num_facilities):
    if pyo.value(model.open_facility[i]) > 0.5:
        print(f"Facility {i}")

print("\nAverage distribution amounts:")
avg_distribution = {(i,j): sum(scenario_probs[s] * pyo.value(model.distribute[i,j,s]) 
                               for s in range(num_scenarios)) 
                    for i in range(num_facilities) for j in range(num_customers)}
for (i,j), amount in avg_distribution.items():
    if amount > 0:
        print(f"From Facility {i} to Customer {j}: {amount:.2f}")

Optimal objective value: 288231.94223017007

Facilities to open:
Facility 0
Facility 2

Average distribution amounts:
From Facility 0 to Customer 1: 1014.30
From Facility 0 to Customer 2: 981.23
From Facility 0 to Customer 3: 1025.61
From Facility 0 to Customer 4: 993.56
From Facility 2 to Customer 0: 991.05
From Facility 2 to Customer 1: 1.09
